In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import re
import preprocessor as prep
from gensim.parsing.preprocessing import remove_stopwords
from nltk import PorterStemmer

def fix(row):
    tweet = str(row['tweet'])
    tweet = re.sub("`", "'", tweet)
    return tweet

def preprocess_tweet(row):
    tweet = str(row['tweet'])
    tweet = prep.clean(tweet)
    return tweet

def stopword_removal(row):
    tweet = str(row['tweet'])
    tweet = remove_stopwords(tweet)
    return tweet

contraction_list = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have","we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(contraction_list.keys()))

def expand_contractions(row, c_re=c_re):
    tweet = str(row['tweet'])
    def replace(match):
        return contraction_list[match.group(0)]
    return c_re.sub(replace, tweet)

def format_tweet(row):
    tweet = str(row['tweet'])
    tweet = tweet.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ')
    return tweet

def stem_tweet(row):
    tweet = str(row['tweet'])
    tweet = PorterStemmer().stem(tweet)
    return tweet

In [3]:
import twint
import nest_asyncio
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.sequence import pad_sequences

from keras.models import load_model
cnn_model = load_model('cnn.h5')

def classify(username):

    nest_asyncio.apply()

    c = twint.Config()
    c.Username = username
    c.Hide_output = True
    c.Limit = 500
    c.Pandas = True
    twint.run.Search(c)
    
    df = twint.storage.panda.Tweets_df
    if(len(df)<10):
        print("Insufficient Data")
        return 
    else:
        df = df[['tweet']]
        df['tweet'] = df.apply(fix, axis=1)
        df['tweet'] = df.apply(preprocess_tweet, axis=1)
        df['tweet'] = df.apply(stopword_removal, axis=1)
        df['tweet'] = df.apply(expand_contractions, axis=1)
        df['tweet'] = df.apply(format_tweet, axis=1)
        df['tweet'] = df.apply(stem_tweet, axis=1)

        with open('tokenizer.pickle', 'rb') as handle:
            tokenizer = pickle.load(handle)
        
        X = df['tweet'].tolist()
        sequence = tokenizer.texts_to_sequences(X)
        data = pad_sequences(sequence, maxlen=120)

        labels_pred = cnn_model.predict(data)
        labels_pred = np.round(labels_pred.flatten())
        total = len(df)
        positive = labels_pred.sum()
        ratio = positive/total
        print(f"Number of tweets obtained : {total}")
        print(f"Fraction of tweets that are indicative of depression : {ratio}")
        if(ratio<=0.02):
            print("At low to no risk of depression")
            return
        elif(ratio<=0.1):
            print("At mild risk of depression")
            return
        elif(ratio<=0.35):
            print("At moderate risk of depression")
            return
        else:
            print("At significant risk of depression")
            return

#### Celebrities with Depression

In [4]:
classify("TheRock")

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.012
At low to no risk of depression


In [5]:
classify('katyperry')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.024
At mild risk of depression


In [6]:
classify('ladygaga')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.034
At mild risk of depression


In [7]:
classify('jk_rowling')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.022
At mild risk of depression


In [8]:
classify('deepikapadukone')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.02
At low to no risk of depression


#### Random Twitter Pages

In [9]:
classify('ndtv')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.018
At low to no risk of depression


In [10]:
classify('IPL')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.0
At low to no risk of depression


In [11]:
classify('bonappetit')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.012
At low to no risk of depression


In [12]:
classify('AnimalPlanet')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.002
At low to no risk of depression


In [13]:
classify('Funfacts')

Number of tweets obtained : 500
Fraction of tweets that are indicative of depression : 0.01
At low to no risk of depression
